<a href="https://colab.research.google.com/github/krishnasameera84/EUR-INR-currency-data-from-Yahoo-Finance/blob/main/EUR_INR_currency_data_from_Yahoo_Finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Installing The Required Libraries**

In [1]:
!pip install yfinance pandas


# **fetching the Data**

In [3]:
import yfinance as yf

ticker = 'EURINR=X'
start_date = '2023-01-01'
end_date = '2024-09-30'

data = yf.download(ticker, start=start_date, end=end_date)

data.to_csv('EURINR_Data_2023_2024.csv')
print("Data saved to 'EURINR_Data_2023_2024.csv'")


[*********************100%***********************]  1 of 1 completed

Data saved to 'EURINR_Data_2023_2024.csv'


# **Download csv file**

In [4]:
from google.colab import files
files.download('EURINR_Data_2023_2024.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


# Load the Data
**First, you need to load the CSV file that you saved previously**

In [6]:
import pandas as pd


date_column_name = 'Date'


try:
    data = pd.read_csv('EURINR_Data_2023_2024.csv', parse_dates=[date_column_name], index_col=date_column_name)
    print("Data loaded successfully.")
    print(data.head())
except ValueError as ve:
    print(f"ValueError: {ve}")
except FileNotFoundError:
    print("File not found. Please check the file path.")
except Exception as e:
    print(f"An error occurred: {e}")


ValueError: Missing column provided to 'parse_dates': 'Date'


In [8]:
print(data.dtypes)


Price      Ticker  
Adj Close  EURINR=X    float64
Close      EURINR=X    float64
High       EURINR=X    float64
Low        EURINR=X    float64
Open       EURINR=X    float64
Volume     EURINR=X      int64
dtype: object


In [9]:

print(data[['Close', 'High', 'Low']].head(10))  # Display the first 10 rows to inspect


Price                          Close       High        Low
Ticker                      EURINR=X   EURINR=X   EURINR=X
Date                                                      
2023-01-02 00:00:00+00:00  88.482002  88.400002  87.157700
2023-01-03 00:00:00+00:00  88.300003  88.313004  87.225998
2023-01-04 00:00:00+00:00  87.219002  87.922997  87.219002
2023-01-05 00:00:00+00:00  87.605003  87.807999  86.757004
2023-01-06 00:00:00+00:00  86.827003  87.614998  86.647003
2023-01-09 00:00:00+00:00  87.500999  88.285004  85.691902
2023-01-10 00:00:00+00:00  88.124001  88.247002  87.445999
2023-01-11 00:00:00+00:00  87.582001  87.935997  87.412003
2023-01-12 00:00:00+00:00  87.805000  88.143997  87.377998
2023-01-13 00:00:00+00:00  88.011002  88.304001  87.726997


In [12]:
# Convert relevant columns to numeric, forcing errors to NaN
# Apply pd.to_numeric to each element in the column
data['Close'] = data['Close'].apply(pd.to_numeric, errors='coerce')
data['High'] = data['High'].apply(pd.to_numeric, errors='coerce')
data['Low'] = data['Low'].apply(pd.to_numeric, errors='coerce')

# Optionally, drop rows with NaN values if needed
data.dropna(inplace=True)

# Calculate Technical Indicators

In [17]:
import pandas as pd
import numpy as np

data = pd.read_csv('EURINR_Data_2023_2024.csv', parse_dates=['Price'], index_col='Price')

data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
data['High'] = pd.to_numeric(data['High'], errors='coerce')
data['Low'] = pd.to_numeric(data['Low'], errors='coerce')

# Optionally drop rows with NaN values if needed
data.dropna(inplace=True)

# Function to calculate moving average
def moving_average(data, window):
    return data['Close'].rolling(window=window).mean()

# Function to calculate Bollinger Bands
def bollinger_bands(data, window=20, num_std=2):
    ma = moving_average(data, window)
    std_dev = data['Close'].rolling(window=window).std()
    upper_band = ma + (std_dev * num_std)
    lower_band = ma - (std_dev * num_std)
    return upper_band, lower_band

# Function to calculate CCI
def commodity_channel_index(data, window=20):
    tp = (data['High'] + data['Low'] + data['Close']) / 3
    ma_tp = tp.rolling(window=window).mean()
    md = tp.rolling(window=window).apply(lambda x: np.mean(np.abs(x - x.mean())), raw=True)
    cci = (tp - ma_tp) / (0.015 * md)
    return cci

# Calculate indicators
data['1-day MA'] = moving_average(data, 1)
data['1-week MA'] = moving_average(data, 7)
data['Upper Band'], data['Lower Band'] = bollinger_bands(data)
data['CCI'] = commodity_channel_index(data)

# Display the last few rows to check the calculations
print(data.tail())


                             Adj Close      Close       High        Low  \
Price                                                                     
2024-09-24 00:00:00+00:00   92.7539978  92.753998  93.248001  92.671997   
2024-09-25 00:00:00+00:00  93.43800354  93.438004  93.627998  93.083000   
2024-09-26 00:00:00+00:00  93.00099945  93.000999  93.467003  92.994003   
2024-09-27 00:00:00+00:00  93.36799622  93.367996  93.629997  93.037003   
2024-09-30 00:00:00+00:00       93.369  93.369000  92.414000  93.840000   

                                  Open Volume   1-day MA  1-week MA  \
Price                                                                 
2024-09-24 00:00:00+00:00  92.77500153      0  92.753998  92.916142   
2024-09-25 00:00:00+00:00  93.49099731      0  93.438004  93.115143   
2024-09-26 00:00:00+00:00  92.99500275      0  93.000999  93.077571   
2024-09-27 00:00:00+00:00   93.3710022      0  93.367996  93.115286   
2024-09-30 00:00:00+00:00       92.414      0  9

<ipython-input-17-1b1365b8dfb9>:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv('EURINR_Data_2023_2024.csv', parse_dates=['Price'], index_col='Price')


In [18]:
data = pd.read_csv('EURINR_Data_2023_2024.csv')
print(data.head())

                       Price    Adj Close        Close         High  \
0                     Ticker     EURINR=X     EURINR=X     EURINR=X   
1                       Date          NaN          NaN          NaN   
2  2023-01-02 00:00:00+00:00  88.48200226  88.48200226  88.40000153   
3  2023-01-03 00:00:00+00:00  88.30000305  88.30000305  88.31300354   
4  2023-01-04 00:00:00+00:00  87.21900177  87.21900177  87.92299652   

           Low         Open    Volume  
0     EURINR=X     EURINR=X  EURINR=X  
1          NaN          NaN       NaN  
2  87.15769958  87.15769958         0  
3  87.22599792  88.29199982         0  
4  87.21900177  87.21900177         0  


In [20]:
data = pd.read_csv('EURINR_Data_2023_2024.csv', parse_dates=['Price'], index_col='Price')

<ipython-input-20-26ac1a145bb6>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv('EURINR_Data_2023_2024.csv', parse_dates=['Price'], index_col='Price')


In [21]:
data = pd.read_csv('EURINR_Data_2023_2024.csv')

## Make Trading Decisions
**Now, based on the calculated indicators, you can analyze the data for September 30, 2024, and make trading decisions:**

In [22]:
import pandas as pd
import numpy as np


data = pd.read_csv('EURINR_Data_2023_2024.csv', parse_dates=['Price'], index_col='Price')


data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
data['High'] = pd.to_numeric(data['High'], errors='coerce')
data['Low'] = pd.to_numeric(data['Low'], errors='coerce')


data.dropna(inplace=True)



# Get the latest data for Sept 30, 2024

data.index = pd.to_datetime(data.index)
# Access the row using .loc:
latest_data = data.loc['2024-09-30']



<ipython-input-22-8b80d4817b37>:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv('EURINR_Data_2023_2024.csv', parse_dates=['Price'], index_col='Price')


# Complete WorkFlow
**After Loading the data, you will recieve the trading decisions based on the calculated technical indicators**

In [24]:
import pandas as pd
import numpy as np

data = pd.read_csv('EURINR_Data_2023_2024.csv', parse_dates=['Price'], index_col='Price')

# Convert 'Close', 'High', and 'Low' columns to numeric
# errors='coerce' will replace invalid values with NaN
data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
data['High'] = pd.to_numeric(data['High'], errors='coerce')
data['Low'] = pd.to_numeric(data['Low'], errors='coerce')

# Optionally drop rows with NaN values if needed:
data.dropna(inplace=True)

# Define functions for the technical indicators
def moving_average(data, window):
    return data['Close'].rolling(window=window).mean()

def bollinger_bands(data, window=20, num_std=2):
    ma = moving_average(data, window)
    std_dev = data['Close'].rolling(window=window).std()
    upper_band = ma + (std_dev * num_std)
    lower_band = ma - (std_dev * num_std)
    return upper_band, lower_band

def commodity_channel_index(data, window=20):
    tp = (data['High'] + data['Low'] + data['Close']) / 3
    ma_tp = tp.rolling(window=window).mean()
    md = tp.rolling(window=window).apply(lambda x: np.mean(np.abs(x - x.mean())), raw=True)
    cci = (tp - ma_tp) / (0.015 * md)
    return cci

# Calculate indicators
data['1-day MA'] = moving_average(data, 1)
data['1-week MA'] = moving_average(data, 7)
data['Upper Band'], data['Lower Band'] = bollinger_bands(data)
data['CCI'] = commodity_channel_index(data)

# Get the latest data for Sept 30, 2024
# Ensure 'Price' column is datetime and access row using .loc:
data.index = pd.to_datetime(data.index)
latest_data = data.loc['2024-09-30']

# Function for making trading decisions (unchanged)
def trading_decision(ma, upper_band, lower_band, cci):
    ma_decision = 'BUY' if ma > latest_data['Close'] else 'SELL'
    bb_decision = 'SELL' if latest_data['Close'] > upper_band else ('BUY' if latest_data['Close'] < lower_band else 'NEUTRAL')
    cci_decision = 'SELL' if cci > 100 else ('BUY' if cci < -100 else 'NEUTRAL')
    return ma_decision, bb_decision, cci_decision

# Get decisions for 1-day and 1-week (unchanged)
decision_1_day = trading_decision(latest_data['1-day MA'], latest_data['Upper Band'], latest_data['Lower Band'], latest_data['CCI'])
decision_1_week = trading_decision(latest_data['1-week MA'], latest_data['Upper Band'], latest_data['Lower Band'], latest_data['CCI'])

print("Trading Decisions for Sept 30, 2024:")
print(f"1-Day Decisions: Moving Average: {decision_1_day[0]}, Bollinger Bands: {decision_1_day[1]}, CCI: {decision_1_day[2]}")
print(f"1-Week Decisions: Moving Average: {decision_1_week[0]}, Bollinger Bands: {decision_1_week[1]}, CCI: {decision_1_week[2]}")

Trading Decisions for Sept 30, 2024:
1-Day Decisions: Moving Average: SELL, Bollinger Bands: NEUTRAL, CCI: NEUTRAL
1-Week Decisions: Moving Average: SELL, Bollinger Bands: NEUTRAL, CCI: NEUTRAL


<ipython-input-24-79da83243f29>:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv('EURINR_Data_2023_2024.csv', parse_dates=['Price'], index_col='Price')
